In [39]:
import numpy as np
import nltk
import glob
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
import pandas as pd
from sklearn.mixture import GMM
from sklearn.metrics import silhouette_score

In [18]:
#Load Shakespeare scenes
f = open('shakespeare_scenes.txt')
shk = f.read()
shakespeare_scenes = eval(shk)
f.close()

#Load Shakespeare acts
f = open('shakespeare_acts.txt')
shk = f.read()
shakespeare_acts = eval(shk)
f.close()

#Create baseline of all text in all plays
all_acts_text = []
all_acts_lines = []
for act in shakespeare_acts:
    all_acts_text.append(act['text'])
all_text = ' '.join(all_acts_text)
all_lines = ' '.join(all_acts_lines)
all_tokens = nltk.word_tokenize(all_text)
all_freq = nltk.FreqDist(all_tokens)
#Get top 20 most frequent terms across all plays
vocabulary = [item[0] for item in sorted(all_freq.items(), key=lambda x: x[1], reverse=True)][:20]

In [19]:
def structural(data):
    elements_text = [element['text'] for element in data]
    elements_lines = [element['lines'] for element in data]
    elements_count = len(data)
    
    features = np.zeros((elements_count, 11), np.float64)
    for i, element in enumerate(elements_lines):
        text = ' '.join(element)
        lines = element
        lines_count = len(lines)
        tokens = nltk.word_tokenize(text.lower())
        sentences = nltk.data.load('tokenizers/punkt/english.pickle').tokenize(text.lower())
        sentences_count = len(sentences)
        words = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text.lower())
        words_unique = list(set(words))        
        words_line_counts = [len(nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(line.lower())) for line in lines]        
        words_sent_counts = [len(nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(sentence.lower())) for sentence in sentences]

        #Calculate features
        mean_word_freq = float(len(words))/float(len(words_unique))
        mean_words_line = np.mean(words_line_counts)
        std_words_line = np.std(words_line_counts)
        mean_words_sent = np.mean(words_sent_counts)
        std_words_sent = np.std(words_sent_counts)
        mean_commas_line = float(tokens.count(","))/float(lines_count)
        mean_commas_sent = float(tokens.count(","))/float(sentences_count)
        mean_colons_line = float(tokens.count(":"))/float(lines_count)
        mean_colons_sent = float(tokens.count(":"))/float(sentences_count)
        mean_scolons_line = float(tokens.count(";"))/float(lines_count)
        mean_scolons_sent = float(tokens.count(";"))/float(sentences_count)
        
        #Assign features to matrix
        features[i,0] = mean_word_freq
        features[i,1] = mean_words_line
        features[i,2] = std_words_line
        features[i,3] = mean_words_sent
        features[i,4] = std_words_sent
        features[i,5] = mean_commas_line
        features[i,6] = mean_commas_sent
        features[i,7] = mean_colons_line
        features[i,8] = mean_colons_sent
        features[i,9] = mean_scolons_line
        features[i,10] = mean_scolons_sent
        
    #scale features
    features = whiten(features)
    return features

In [34]:
def word_freq(data):
    elements_text = [element['text'] for element in data]
    elements_lines = [element['lines'] for element in data]
    elements_count = len(data)
    
    vectorizer = CountVectorizer(vocabulary=vocabulary, tokenizer=nltk.word_tokenize)
    freq_vec = vectorizer.fit_transform(elements_text).toarray().astype(np.float64)
    freq_vec /= np.c_[np.apply_along_axis(np.linalg.norm, 1, freq_vec)]
 
    return freq_vec

In [48]:
def authorship(data, clusters):
    freq = word_freq(data)
    struc = structural(data)
    km_freq = KMeans(n_clusters=clusters, init='k-means++', n_init=99)
    km_struc = KMeans(n_clusters=clusters, init='k-means++', n_init=99)
    km_freq.fit(freq)
    km_struc.fit(struc)
    km_freq_pred = km_freq.predict(freq)
    km_struc_pred = km_struc.predict(struc)
    km_freq_score = silhouette_score(freq, km_freq_pred, metric='sqeuclidean')
    km_struc_score = silhouette_score(struc, km_struc_pred, metric='sqeuclidean')
    gmm_freq = GMM(n_components = clusters, covariance_type="full")
    gmm_struc = GMM(n_components = clusters, covariance_type="full")
    gmm_freq.fit(freq)
    gmm_struc.fit(struc)
    gmm_freq_pred = km_freq.predict(freq)
    gmm_struc_pred = km_struc.predict(struc)
    gmm_freq_score = silhouette_score(freq, gmm_freq_pred, metric='sqeuclidean')
    gmm_struc_score = silhouette_score(freq, gmm_struc_pred, metric='sqeuclidean')
    return [km_freq, km_freq_score], [km_struc, km_struc_score], [gmm_freq, gmm_freq_score], [gmm_struc, gmm_struc_score]

In [49]:
km_acts_freq, km_acts_struc, gmm_acts_freq, gmm_acts_struc = authorship(shakespeare_acts, 3)
km_scenes_freq, km_scenes_struc, gmm_scenes_freq, gmm_scenes_struc = authorship(shakespeare_scenes, 3)

print ("Acts Scores:")
print("Frequency: gmm:", gmm_acts_freq[1], "km:", km_acts_freq[1])
print("Structural: gmm:", gmm_acts_struc[1], "km:", km_acts_struc[1])
print ("Scenes Scores:")
print("Frequency: gmm:", gmm_scenes_freq[1], "km:", km_scenes_freq[1])
print("Structural: gmm:", gmm_scenes_struc[1], "km:", km_scenes_struc[1])

Acts Scores:
Frequency: gmm: 0.287220120083 km: 0.287220120083
Structural: gmm: 0.207194553257 km: 0.329497252899
Scenes Scores:
Frequency: gmm: 0.210005008694 km: 0.210005008694
Structural: gmm: 0.145588370851 km: 0.411959480389


C:\Users\greeniend\Anaconda\envs\py3k\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [52]:
columns = ['Title', 'Act', 'Frequency_km', 'Structural_km']
clusters = pd.DataFrame(columns = columns)
clusters['Title'] = [item['title'] for item in shakespeare_acts]
clusters['Act'] = [item['act'] for item in shakespeare_acts]
clusters['Frequency_km'] = km_acts_freq[0].labels_
clusters['Structural_km'] = km_acts_struc[0].labels_
print (clusters)
clusters.to_csv('clusters_acts.csv')

                            Title Act  Frequency_km  Structural_km
0       All's Well That Ends Well   1             1              0
1       All's Well That Ends Well   2             1              1
2       All's Well That Ends Well   3             1              1
3       All's Well That Ends Well   4             1              1
4       All's Well That Ends Well   5             1              1
5                  As You Like It   1             1              1
6                  As You Like It   2             0              0
7                  As You Like It   3             1              1
8                  As You Like It   4             1              1
9                  As You Like It   5             1              0
10           The Comedy of Errors   1             0              0
11           The Comedy of Errors   2             1              1
12           The Comedy of Errors   3             1              1
13           The Comedy of Errors   4             1           

In [53]:
columns = ['Title', 'Act', 'Frequency_km', 'Structural_km']
clusters = pd.DataFrame(columns = columns)
clusters['Title'] = [item['title'] for item in shakespeare_scenes]
clusters['Act'] = [item['act'] for item in shakespeare_scenes]
clusters['Scene'] = [item['scene'] for item in shakespeare_scenes]
clusters['Frequency_km'] = km_scenes_freq[0].labels_
clusters['Structural_km'] = km_scenes_struc[0].labels_
print (clusters)
clusters.to_csv('clusters_scenes.csv')

                         Title Act  Frequency_km  Structural_km Scene
0    All's Well That Ends Well   1             0              2     1
1    All's Well That Ends Well   1             1              0     2
2    All's Well That Ends Well   1             0              0     3
3    All's Well That Ends Well   2             0              0     1
4    All's Well That Ends Well   2             0              2     2
5    All's Well That Ends Well   2             0              2     3
6    All's Well That Ends Well   2             0              2     4
7    All's Well That Ends Well   2             0              2     5
8    All's Well That Ends Well   3             1              2     1
9    All's Well That Ends Well   3             0              2     2
10   All's Well That Ends Well   3             1              2     3
11   All's Well That Ends Well   3             1              0     4
12   All's Well That Ends Well   3             0              2     5
13   All's Well That